In [168]:
import cv2
import numpy as np
import time
import sys
import pickle

select_pts=0 #1 to select--> you have to run the other selection scripts first
if select_pts==1:
    reward_pts= pickle.load(sys.stdin)
else:
     reward_pts = [[(250, 125), (290, 200)],[(330, 125), (370, 200)],[(250, 315), (290, 390)], [(330, 315), (370, 390)]]


# function to collect the differences between frames to later decide if there was any movement in the reward areas.
def diffImg(t0, t1, t2):
    d1 = cv2.absdiff(t2, t1)
    d2 = cv2.absdiff(t1, t0)
    return cv2.bitwise_and(d1, d2)


# Same command function as streaming, its just now we pass in the file path, nice!
cap = cv2.VideoCapture('C:/Users/isabe/Downloads/mice_movie.mp4') # add a 0 to connect to the camera 
time.sleep(2)# give it time to load

# FRAMES PER SECOND FOR VIDEO
fps = 60

# Always a good idea to check if the video was acutally there
# If you get an error at thsi step, triple check your file path!!
if cap.isOpened()== False: 
    print("Error opening the video file. Please double check your file path for typos. Or move the movie file to the same location as this script/notebook")
    
        
##
mice_loc=np.zeros(len(reward_pts))
# Create a kernel
kernel=np.ones((5,5), np.uint8)
#cv2.namedWindow('Test')

first_frame=None 
ret, frame = cap.read()
    
  #While the video is opened
while cap.isOpened():
    
        
    # Read the video file.
    ret, frame = cap.read()
    
    # If we got frames, show them.
    if ret == True:
         
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)#convert to gray scale
        gray = cv2.GaussianBlur(gray, (21,21),0)# blur image for better object detection
        if first_frame is None:
            first_frame=gray
            continue
        
        delta_frame = cv2.absdiff(first_frame,gray) # get only the stuck that is not static 
        thresh_frame = cv2.threshold(delta_frame, 80, 255, cv2.THRESH_BINARY)[1] #binary threshold
        thresh_frame = cv2.morphologyEx(thresh_frame, cv2.MORPH_GRADIENT, kernel)
         
            
            #find the mouse 
        (cnts,hierarchy) = cv2.findContours(thresh_frame.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        
        #draw around the mouse 
        for contour in cnts:
            #if cv2.contourArea(contour) < 4000:
             #   continue

            (x,y, w, h)= cv2.boundingRect(contour)
            cv2.rectangle(frame, (x, y), (x + w, y + h),(0,225,0),2)
            
        #find out in mouse is in the rward area 
        for i in range (0, len(cnts)):
            (x,y, w, h)= cv2.boundingRect(cnts[i])
            xCentre=int(x+0.5*w)
            yCentre=int(y+0.5*h)
            cv2.circle(img=frame,center=(xCentre,yCentre),radius=5,color=(255,0,0),thickness=2)  
            result=np.zeros(len(reward_pts))    
            for r in range(0, len(reward_pts)): #len(reward_pts)
                #rewardArea= cv2.rectangle(thresh_frame,pt1=reward_pts[i][0],pt2=reward_pts[i][1],color=(255,0,0),thickness=2)
                rewardContour=np.array([[1,1],reward_pts[r][0],reward_pts[r][1]],  dtype=np.int32)

            
            #### Check if the point was in the reward area: 
                result[r] = cv2.pointPolygonTest(rewardContour, (xCentre,yCentre), False)
          
        
        # save in the array the result of whether the mouse is in or out
        mice_loc=np.vstack([mice_loc,result])# a
        #cv2.imshow("Delta Frame",delta_frame)
       # cv2.imshow("Capturing",gray)
        cv2.imshow("Threshold Frame",thresh_frame) # to see the object detection
        cv2.imshow("Color Frame",frame) # actual video 

        key = cv2.waitKey(5)
        
        
         # Display the frame at same frame rate of recording
        time.sleep(1/fps)# only needed for video- not camera
        
        
     
       # cv2.imshow('Test',frame)
 
        # Press q to quit
        if cv2.waitKey(25) & 0xFF == ord('q'):
            
            break
            
      
    # Or automatically break this whole loop if the video is over.
    else:
        break
        
cap.release()
# Closes all the frames
cv2.destroyAllWindows()


mice_loc.tofile('test.dat')

In [157]:
       
cap.release()
# Closes all the frames
cv2.destroyAllWindows()


In [169]:
mice_loc.shape

(526, 4)

(480, 640)

numpy.ndarray

array([], dtype=float64)